In [1]:
import codecs,unidecode,yaml,glob,os,shutil,re
import itertools as it
import numpy as np
import pandas as pd
# print(pd.__version__)

In [2]:
nQuizz=7
fMajeure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Majeure.csv"
fMineure=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Mineure.csv"
fCM=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/CM.csv"
fAliasQDLE=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz-Alias.yaml"
fQuizzoodle=u"/Users/gilles/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/01-NotesQuizz/Quizz%02d.csv"%nQuizz
fNotes=fQuizzoodle.replace(u".csv",u"-Notes.csv")

In [3]:
'''https://stackoverflow.com/a/35491326/3986209'''
def invert_dict(d): 
    inverse = dict() 
    for key in d: 
        # Go through the list that is saved in the dict:
        for item in d[key]:
            # Check if in the inverted dict the key exists
            if item not in inverse: 
                # If not create a new list
                inverse[item] = [key] 
            else: 
                inverse[item].append(key) 
    return inverse

# Lecture des données

In [4]:
try:
    with codecs.open(fAliasQDLE, 'r', encoding="utf8") as inFile:
        correspondances=yaml.safe_load(inFile)
    alias=correspondances["alias2etu"]
except:
    alias={}
alias

{'ALINA': ['433401'],
 'Agathe / Liliana': ['430440', '430952'],
 'Agathe Martin': ['430440'],
 'Alex Helvert': ['380824'],
 'Alice LOOF': ['427886'],
 'Alima Gassama': ['418366'],
 'Alissa Bessettes': ['429623'],
 'Alix DEBUR': ['427449'],
 u'Ali\xe9nor Riou': ['428930'],
 'Amandine B': ['429252'],
 'Amelie Mitlehner': ['433485'],
 u'Am\xe9lie rochefort': ['432360'],
 u'Ana\xeblle DINA-SAME': ['432151'],
 'Auriault justin': ['430711'],
 'Axel Meunier': ['429855'],
 'BENTO Elena': ['428999'],
 u'BOUSSARD L\xe9a': ['428452'],
 'Baptiste phelip': ['397583'],
 u'Barsacq Th\xe9o': ['403123'],
 'Berloin Amandine': ['429252'],
 'Bertrand Sascha': ['419149'],
 'Blanc Sheryl': ['427234'],
 'Bonnet Nathan': ['433074'],
 u'Bordelais Cl\xe9a': ['429396'],
 u'Bordelais Cl\xe9a ettouil sahra': ['431767', '429396'],
 'CHADDOU hilel': ['430892'],
 'CLOCHET': ['427924'],
 'CLOCHET LYSON': ['427924'],
 'CLOCHET lyson': ['427924'],
 'Camille Villechalanne': ['429928'],
 'Carla': ['433437'],
 'Chaigne El

In [5]:
len(alias)

211

In [6]:
colCM=[u"id",u"nom",u"prénom",u"email",u"ine"]
colQuizz=["pseudo","points","temps"]

try:
    dfCM=pd.read_csv(filepath_or_buffer=fCM,sep="\t",encoding="utf8",header=0,names=colCM)
    lQuizz=pd.read_csv(filepath_or_buffer=fQuizzoodle,sep=";",encoding="utf8",header=0,names=colQuizz)
    lQuizz.pseudo=lQuizz.pseudo.str.strip()
except:
    dfCM=None
    lQuizz=None

In [7]:
# lQuizz, dfCM

# Listage des alias

In [8]:
dfCM["PN"]=dfCM[u"prénom"].str.lower().str.strip()+" "+dfCM[u"nom"].str.lower().str.strip()
dfCM["NP"]=dfCM[u"nom"].str.lower().str.strip()+" "+dfCM[u"prénom"].str.lower().str.strip()
dfCM

,id,nom,prénom,email,ine,PN,NP
0,415923,AKKUL,EMMANUEL,emmanuel.akkul@etu.u-bordeaux-montaigne.fr,1664938189,emmanuel akkul,akkul emmanuel
1,432822,AUBERT,EMMA,emma.aubert@etu.u-bordeaux-montaigne.fr,1664938189,emma aubert,aubert emma
2,430711,AURIAULT,JUSTIN,justin.auriault@etu.u-bordeaux-montaigne.fr,1664938189,justin auriault,auriault justin
3,431129,BARANGER,MANON,manon.baranger@etu.u-bordeaux-montaigne.fr,1664938189,manon baranger,baranger manon
4,403123,BARSACQ,THEO,theo.barsacq@etu.u-bordeaux-montaigne.fr,1664938189,theo barsacq,barsacq theo
5,433205,BEFORT,KRISTIN,kristin.befort@etu.u-bordeaux-montaigne.fr,1664938189,kristin befort,befort kristin
6,428999,BENTO,ELENA,elena.bento@etu.u-bordeaux-montaigne.fr,1664938189,elena bento,bento elena
7,429252,BERLOIN,AMANDINE,amandine.berloin@etu.u-bordeaux-montaigne.fr,1664938189,amandine berloin,berloin amandine
8,419149,BERTRAND,SASCHA,sascha.bertrand@etu.u-bordeaux-montaigne.fr,1664938189,sascha bertrand,bertrand sascha
9,429623,BESSETTES,ALISSA,alissa.bessettes@etu.u-bordeaux-montaigne.fr,1664938189,alissa bessettes,bessettes alissa


In [9]:
pseudos=lQuizz[~(lQuizz["pseudo"].isin(alias))]["pseudo"].str.strip().tolist()
print "\n".join(pseudos)

Mathou
SHA Yel
Frechet-Klopstein
Karaketir Sila
chaigne elise
Liliana / Agathe
DELAHODDE EVA
Zeubi
Emma-Lou Pohu
GRAFFE
Elysa Ravet Mariah Gotter
K


In [10]:
for nP,pseudoBrut in enumerate(pseudos):
    pseudo=unidecode.unidecode(pseudoBrut)
    print pseudo
    sPseudo = raw_input("Nom a chercher:\n")
    if sPseudo:
        print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]
        idPseudo=raw_input("id :\n")
        if idPseudo:
            alias[pseudoBrut]=idPseudo

Mathou
Nom a chercher:
math
                   PN      id
20   mathilde breuils  415694
103  lenaelle mathieu  417374
112   mathis monteiro  427574
121  mathilde prevost  432779
id :

SHA Yel
Nom a chercher:
yel
                      PN      id
140  yelyzaveta shadrina  425485
id :
425485
Frechet-Klopstein
Nom a chercher:
frech
               PN      id
62  manon frechet  429499
id :
428316 429499
Karaketir Sila
Nom a chercher:
kara
                PN      id
81  sila karaketir  427702
id :
427702
chaigne elise
Nom a chercher:
chaigne
               PN      id
29  elise chaigne  429273
id :
chaigne
Liliana / Agathe
Nom a chercher:
liliana
                        PN      id
32  liliana correia santos  430952
id :
430440 430952
DELAHODDE EVA
Nom a chercher:
delah
               PN      id
38  eva delahodde  427303
id :
427303
Zeubi
Nom a chercher:
zeu
Empty DataFrame
Columns: [PN, id]
Index: []
id :

Emma-Lou Pohu
Nom a chercher:
pohu
                PN      id
118  emma-lou pohu  426820

In [11]:
# pseudo=u"csaintavi001 ebismes001"
# alias[pseudo]=["415962","413429"]

In [12]:
sPseudo="bisme"
print dfCM[dfCM["PN"].str.contains(sPseudo.lower())][["PN","id"]]

Empty DataFrame
Columns: [PN, id]
Index: []


In [13]:
pseudos

[u'Mathou',
 u'SHA Yel',
 u'Frechet-Klopstein',
 u'Karaketir Sila',
 u'chaigne elise',
 u'Liliana / Agathe',
 u'DELAHODDE EVA',
 u'Zeubi',
 u'Emma-Lou Pohu',
 u'GRAFFE',
 u'Elysa Ravet Mariah Gotter',
 u'K']

In [14]:
# alias["Sofiane"]

In [15]:
pn=dfCM["PN"].tolist()
np=dfCM["NP"].tolist()

In [16]:
for pseudo in pseudos:
    if unidecode.unidecode(pseudo.lower()) in pn:
        print pseudo,dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["PN"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]
    elif unidecode.unidecode(pseudo.lower()) in np:
        print pseudo,dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0]
        alias[pseudo]=[str(dfCM[dfCM["NP"]==unidecode.unidecode(pseudo.lower())]["id"].values[0])]



Karaketir Sila 427702
chaigne elise 429273
DELAHODDE EVA 427303
Emma-Lou Pohu 426820


In [17]:
for al in alias:
    if not isinstance(alias[al],list):
        if " " in alias[al]:
            alias[al]=alias[al].split(" ")
        else:
            alias[al]=[alias[al]]

In [18]:
len(alias)

220

In [19]:
invAlias=invert_dict(alias)
#invAlias[385732]=[u'adiarra001, cbeauvais004']
correspondances={"alias2etu":alias,"etu2alias":invAlias}
correspondances

{'alias2etu': {'ALINA': ['433401'],
  'Agathe / Liliana': ['430440', '430952'],
  'Agathe Martin': ['430440'],
  'Alex Helvert': ['380824'],
  'Alice LOOF': ['427886'],
  'Alima Gassama': ['418366'],
  'Alissa Bessettes': ['429623'],
  'Alix DEBUR': ['427449'],
  u'Ali\xe9nor Riou': ['428930'],
  'Amandine B': ['429252'],
  'Amelie Mitlehner': ['433485'],
  u'Am\xe9lie rochefort': ['432360'],
  u'Ana\xeblle DINA-SAME': ['432151'],
  'Auriault justin': ['430711'],
  'Axel Meunier': ['429855'],
  'BENTO Elena': ['428999'],
  u'BOUSSARD L\xe9a': ['428452'],
  'Baptiste phelip': ['397583'],
  u'Barsacq Th\xe9o': ['403123'],
  'Berloin Amandine': ['429252'],
  'Bertrand Sascha': ['419149'],
  'Blanc Sheryl': ['427234'],
  'Bonnet Nathan': ['433074'],
  u'Bordelais Cl\xe9a': ['429396'],
  u'Bordelais Cl\xe9a ettouil sahra': ['431767', '429396'],
  'CHADDOU hilel': ['430892'],
  'CLOCHET': ['427924'],
  'CLOCHET LYSON': ['427924'],
  'CLOCHET lyson': ['427924'],
  'Camille Villechalanne': ['4

In [20]:
with codecs.open(fAliasQDLE, 'w', encoding="utf8") as outfile:
    yaml.safe_dump(correspondances, outfile, default_flow_style=False)

# Calcul des notes
Attention au nombre de questions

In [21]:
nbQ=7.0

In [22]:
def aliasQuizz(pseudo):
    result=None
    if pseudo in alias:
        result=alias[pseudo]
    return result

def noteQuizz(points):   
    result=8
    if points>0:
        result=int(round(10*(1+points/nbQ),0))
    return result
        
        

lQuizz["id"]=lQuizz["pseudo"].apply(aliasQuizz)
lQuizz["note"]=lQuizz["points"].apply(noteQuizz)

In [23]:
def splitPseudo1(id):
    result=None
    if id and len(id)>0:
        result=id[0]
    return result

def splitPseudo2(id):
    result=None
    if id and len(id)>1:
        result=id[1]
    return result

def splitPseudo3(id):
    result=None
    if id and len(id)>2:
        result=id[2]
    return result

lQuizz["id1"]=lQuizz["id"].apply(splitPseudo1)
lQuizz["id2"]=lQuizz["id"].apply(splitPseudo2)
lQuizz["id3"]=lQuizz["id"].apply(splitPseudo3)

lQuizz

,pseudo,points,temps,id,note,id1,id2,id3
0,Vadim Olivan,6,234.42,[414257],19,414257,None,None
1,Romain Dubois,6,239.89,[417953],19,417953,None,None
2,Louann le corf,6,271.51,[431735],19,431735,None,None
3,Labille romane,5,201.39,[431590],17,431590,None,None
4,BENTO Elena,5,252.70,[428999],17,428999,None,None
5,Elianemegarus,5,265.87,[413748],17,413748,None,None
6,Laëtitia Capelo,5,270.85,[427987],17,427987,None,None
7,ROGER Matisse,5,278.12,[433263],17,433263,None,None
8,Dehoulle Fanny,5,278.87,[396374],17,396374,None,None
9,Camille Villechalanne,5,283.61,[429928],17,429928,None,None


In [24]:
df1=lQuizz[["pseudo","note","temps","id1"]].dropna()
df2=lQuizz[["pseudo","note","temps","id2"]].dropna()
df3=lQuizz[["pseudo","note","temps","id3"]].dropna()
df1.columns=["pseudo","note","temps","id"]
df2.columns=["pseudo","note","temps","id"]
df3.columns=["pseudo","note","temps","id"]

In [25]:
df123=pd.concat([df1,df2,df3]).sort_values(["note","temps"],ascending=[False,True])
df123

,pseudo,note,temps,id
0,Vadim Olivan,19,234.42,414257
1,Romain Dubois,19,239.89,417953
2,Louann le corf,19,271.51,431735
3,Labille romane,17,201.39,431590
4,BENTO Elena,17,252.70,428999
5,Elianemegarus,17,265.87,413748
6,Laëtitia Capelo,17,270.85,427987
7,ROGER Matisse,17,278.12,433263
8,Dehoulle Fanny,17,278.87,396374
9,Camille Villechalanne,17,283.61,429928


In [34]:
df123.loc[(df123.note==8)&(df123.temps>0),"note"]=10

In [35]:
df123.at[0, 'note'] = 20
if df123.loc[1, "note"]<18:
    df123.at[1, 'note'] = 18
df123

,pseudo,note,temps,id
0,Vadim Olivan,20,234.42,414257
1,Romain Dubois,19,239.89,417953
2,Louann le corf,19,271.51,431735
3,Labille romane,17,201.39,431590
4,BENTO Elena,17,252.70,428999
5,Elianemegarus,17,265.87,413748
6,Laëtitia Capelo,17,270.85,427987
7,ROGER Matisse,17,278.12,433263
8,Dehoulle Fanny,17,278.87,396374
9,Camille Villechalanne,17,283.61,429928


In [36]:
dfNotes=df123.groupby("id").max().sort_values("note",ascending=False)
dfNotes.to_csv(fNotes,sep="\t",encoding="utf8")